In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/MyProject/dataset/data.csv')

FileNotFoundError: ignored

<hr>

# Data Preprocessing

## 1 - look at data

Let’s see the dimensions of the dataframe.

In [ ]:
df.shape

(99003, 12)

<hr>

In [ ]:
df.head()

,userid,age,gender,tenure,friend_count,friendships_initiated,likes,likes_received,mobile_likes,mobile_likes_received,www_likes,www_likes_received
0,2094382,14,male,266.0,0,0,0,0,0,0,0,0
1,1192601,14,female,6.0,0,0,0,0,0,0,0,0
2,2083884,14,male,13.0,0,0,0,0,0,0,0,0
3,1203168,14,female,93.0,0,0,0,0,0,0,0,0
4,1733186,14,male,82.0,0,0,0,0,0,0,0,0


<hr>

## 2 - Remove Unnecessary Columns

We need all the columns for this project.

<hr>

## 3 - Change Column’s Name

I change the name of the column based on the following:
* friend_count: friends
* friendships_initiated: request
* likes: g_likes
* likes_received: r_likes

In [ ]:
col_name = ['userid', 'age', 'gender', 'tenure', 'friends',
       'request', 'g_likes', 'r_likes', 'mobile_likes',
       'mobile_likes_received', 'www_likes', 'www_likes_received']

df.columns = col_name

print(df.columns)

Index(['userid', 'age', 'gender', 'tenure', 'friends', 'request', 'g_likes',
       'r_likes', 'mobile_likes', 'mobile_likes_received', 'www_likes',
       'www_likes_received'],
      dtype='object')


<hr>

## 4 - Handle Missing Values

I create a for loop to iterate among all columns to realize whether they have null values or not. I’m looking for the number of null values in every single column as well as the percentage of null values.

In [ ]:
for col in df.columns:
    number_null = df.loc[: , col].isnull().sum()
    perc_null = (number_null / df.shape[0]) * 100
    print('{} - {} - %{}'.format(col, number_null, round(perc_null,5)))

userid - 0 - %0.0
age - 0 - %0.0
gender - 175 - %0.17676
tenure - 2 - %0.00202
friends - 0 - %0.0
request - 0 - %0.0
g_likes - 0 - %0.0
r_likes - 0 - %0.0
mobile_likes - 0 - %0.0
mobile_likes_received - 0 - %0.0
www_likes - 0 - %0.0
www_likes_received - 0 - %0.0


We must have a different approach to handling null values. Since I have a large dataset:
<br><br>
Categorical:
* less than 5%, I drop the rows.
* between 5% and 30%, I impute with mode.
* More than 30%, create a new label as “Other.”
<br><br>

Numerical:
* between 0% and 30%, I impute with mean or median.
* More than 30%, I drop the rows.

<br>
However, the best way is consulting with expert domain. 

<hr>

Let’s begin with categorical variables. First, I deal with <b>“gender”</b> that has %0.177 null values. since it is less than 5%, I drop them.

In [ ]:
df = df.dropna(subset = ['gender'])

<hr>

And now, I work on numerical variables. I work on <b>“tenure”</b> and since it's less than 30%, I should impute it. But before doing this, I must make sure about distribution shape of these columns to see whether they are right-skewed or left-skewed. It can be helpful when I want to decide choosing mean or median for imputing. Also, I should check the data type to be sure about numerical type.

In [ ]:
print(df['tenure'].dtypes)

float64


the result shows this variable has correct data type.

Now we can see its distribution.

In [ ]:
print('Skewness :' , round(df['tenure'].skew() ,3))

mean_tenure = df['tenure'].mean()
median_tenure = df['tenure'].median()

if mean_tenure > median_tenure:
    print('Mean is bigger than Median. Left Skewed. Median for imputing')
else:
    print('Mean is smaller than Median. Right Skewed. Mean for imputing')

Skewness : 1.531
Mean is bigger than Median. Left Skewed. Median for imputing


The result shows that I should choose median for imputing.

In [ ]:
df['tenure'] = df['tenure'].fillna(median_tenure).round(0)

Finally, we check the null values for dataset again.

In [ ]:
for col in df.columns:
    number_null = df.loc[: , col].isnull().sum()
    perc_null = (number_null / df.shape[0]) * 100
    print('{} - {} - %{}'.format(col, number_null, round(perc_null,5)))

userid - 0 - %0.0
age - 0 - %0.0
gender - 0 - %0.0
tenure - 0 - %0.0
friends - 0 - %0.0
request - 0 - %0.0
g_likes - 0 - %0.0
r_likes - 0 - %0.0
mobile_likes - 0 - %0.0
mobile_likes_received - 0 - %0.0
www_likes - 0 - %0.0
www_likes_received - 0 - %0.0


<hr>

## 5 - Handle Duplicate Rows

Now we should handle duplicate rows. Since all values might be same, we just we need to check whether there are two rows that all values in all columns are the same or not.

In [ ]:
duplicate_rows = df.duplicated()

if duplicate_rows.any():
    print("The DataFrame has duplicate rows.")
else:
    print("The DataFrame does not have duplicate rows.")

The DataFrame does not have duplicate rows.


<hr>

## 6 - Handle Number Variables

First of all, I declare all number variables. 

In [ ]:
num_list = ['age', 'tenure', 'friends','request', 'g_likes', 'r_likes', 'mobile_likes', 'mobile_likes_received', 'www_likes',
       'www_likes_received']

I must make sure about the data type of the number variable. Just because the column shows numbers, it doesn't mean that they are numbers.Thus, with regular expression I should clean them.

In [ ]:
def non_numeric(x):
    non_numeric_df = pd.DataFrame(df[df[x].astype(str).str.contains('[^\d\.]+')])
    return non_numeric_df

In [ ]:
non_numeric('age')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('tenure')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('friends')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('request')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('g_likes')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('r_likes')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('mobile_likes')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('mobile_likes_received')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('www_likes')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


In [ ]:
non_numeric('www_likes_received')

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received


The results show fortunately in those variables, we just have number nothing else. However, we can see the sum of those variables for double check.

In [ ]:
for i in range(len(num_list)):
    var_sum = df.loc[: , num_list[i]].sum()
    print(num_list[i] , var_sum)

age 3677768
tenure 52936947.0
friends 19407045
request 10621917
g_likes 15428029
r_likes 14099133
mobile_likes 10490184
mobile_likes_received 8313152
www_likes 4937840
www_likes_received 5785977


In the next, we should run sanity check. In this dataset, <b>“g_likes”</b> must be aggregation of <b>“mobile_likes”</b> and <b>“ww_likes”</b>. Also, for <b>“r_likes”</b> we have the same approach. We check this matter and if is there any mismatched data, we consider new columns as getting like and receiving likes.

In [ ]:
df['sanity_g_like'] = df['mobile_likes'] + df['www_likes']
df['diff_g_like'] = df['g_likes'] - df['sanity_g_like']

Now we should check whether there is any difference or not. 

In [ ]:
df['diff_g_like'].sum()

5

It means there are 5 records that sum of <b>“mobile_likes”</b> and <b>“ww_likes”</b> does not equal to <b>“g_likes”</b>.  In this case, we should ignore the old one, and consider the new column’s value.

In [ ]:
df.sort_values(by = 'diff_g_like', ascending = False)

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received,sanity_g_like,diff_g_like
27607,1930804,41,female,782.0,35,14,176,159,174,142,0,17,174,2
93007,1535515,23,female,510.0,629,440,1298,1522,1216,1172,81,350,1297,1
90002,1030735,21,male,373.0,473,409,896,136,895,113,0,23,895,1
98827,1182272,17,female,1082.0,4464,1716,2049,17159,681,9657,1367,7502,2048,1
65997,2130046,21,female,592.0,146,66,134,94,134,27,0,67,134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32994,1940245,53,male,544.0,45,28,12,45,12,21,0,24,12,0
32993,1639797,28,female,348.0,45,34,12,22,12,12,0,10,12,0
32992,2142529,51,male,1256.0,45,13,12,6,12,3,0,3,12,0
32991,1353186,18,male,210.0,44,12,12,2,12,1,0,1,12,0


In [ ]:
df.loc[df['diff_g_like'] > 0, 'g_likes'] = df['sanity_g_like']

Now, we run the same process for receiving like.

In [ ]:
df['sanity_r_like'] = df['mobile_likes_received'] + df['www_likes_received']
df['diff_r_like'] = df['r_likes'] - df['sanity_r_like']
df['diff_r_like'].sum()

4

In [ ]:
df.sort_values(by = 'diff_r_like', ascending = False)

,userid,age,gender,tenure,friends,request,g_likes,r_likes,mobile_likes,mobile_likes_received,www_likes,www_likes_received,sanity_g_like,diff_g_like,sanity_r_like,diff_r_like
83113,1670016,14,female,32.0,293,174,3423,2222,2581,1847,842,374,3423,0,2221,1
98981,1277820,17,female,1005.0,4351,2210,5045,4498,4742,3909,303,588,5045,0,4497,1
96931,1698567,19,female,600.0,1028,754,1916,1292,1672,722,244,569,1916,0,1291,1
90680,2088542,80,female,1704.0,501,221,138,65,99,55,39,9,138,0,64,1
65998,1411998,38,male,616.0,146,87,134,104,134,50,0,54,134,0,104,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32993,1639797,28,female,348.0,45,34,12,22,12,12,0,10,12,0,22,0
32992,2142529,51,male,1256.0,45,13,12,6,12,3,0,3,12,0,6,0
32991,1353186,18,male,210.0,44,12,12,2,12,1,0,1,12,0,2,0
32990,1248235,46,male,455.0,44,26,12,0,12,0,0,0,12,0,0,0


Then, I change them to the correct value.

In [ ]:
df.loc[df['diff_r_like'] > 0, 'r_likes'] = df['sanity_r_like']

Finally, I remove the extra columns.

In [ ]:
df = df.drop(['sanity_g_like', 'diff_g_like', 'sanity_r_like','diff_r_like', 
              'mobile_likes', 'mobile_likes_received', 
              'www_likes', 'www_likes_received'], axis = 1)

<hr>

## 7 - Handle Categorical Variables

Now, I declare all categorical variables. 

In [ ]:
cat_list = ['gender']

Then, we must make sure about the possible range for each of them. They must be the same with data documentation. 

> ### gender

In [ ]:
 print(df['gender'].unique())

['male' 'female']


In [ ]:
 print(df['gender'].value_counts())

male      58574
female    40254
Name: gender, dtype: int64


All of them are correct and based on data documentation. 

<hr>

# Check Point

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
import pickle
with open('dataset/df_preprocessed.pickle', 'wb') as file:
    pickle.dump(df, file)